# Cashflow management problem

Details of the problem are as follows:
+ Liability = [150.0,100.0,-200.0,200.0,-50.0,-350.0]
+ Financial Instruments
    + Line of credit: Limit 100K, interest = 1% per month
    + Commercial paper: Unlimited, duration = 3 months, interest = 2% for 3 months
    + Bank account: Unlimited, interest = 0.3% per month

Import the relevant python modules

In [1]:
import numpy as np
from tabulate import *
import cvxpy as cvx

Set up the data for the problem

In [2]:
liability = np.array([150.0,100.0,-200.0,200.0,-50.0,-350.0])
ntimes = len(liability);
paper_period = 3;
paper_int = 2;
rf_int = 0.3;
loan_int = 1;
loan_limit = 100;


Set up the variables 

In [3]:
x = cvx.Variable(ntimes-1)
y = cvx.Variable(ntimes-paper_period)
z = cvx.Variable(ntimes)

Set up the constraints for each time period

In [4]:
constraints = [];
# flow constraints for each time period
for t in range(ntimes):
    expr = []
    if (t>0):
        expr += [-(1+loan_int/100)*x[t-1]];
    if (t<ntimes-1):
        expr += [x[t]]
    if (t>paper_period-1):
        expr += [-(1+paper_int/100)*y[t-paper_period]];
    if (t<ntimes-paper_period):
        expr += [y[t]];
    if (t>0):
        expr += [(1+rf_int/100)*z[t-1]];
    expr += [-z[t]];
    constraints += [sum(expr) >= liability[t]];

# upper bound on the line of credit
constraints += [x <= loan_limit]

# non-negativity constraints
constraints += [x>=0, y>=0, z>=0];

# objective function
objective = cvx.Maximize(z[ntimes-1]);

Set up the optimization and solve it using the default cvxpy solver

In [5]:
prob = cvx.Problem(objective, constraints)
prob.solve()

np.set_printoptions(precision=4,suppress=True)
print('Problem status: ' + str(prob.status));
if (prob.status == 'optimal'):
    print('Problem value: %.3f' % prob.value);
    print('loan values: ')
    print(x.value.transpose())
    print('paper values: ')
    print(y.value.transpose())
    print('cash values: ')
    print(z.value.transpose())

Problem status: optimal
Problem value: 142.497
loan values: 
[ 0.     24.7896  0.      0.     26.7146]
paper values: 
[150.      75.2104 176.9816]
cash values: 
[  0.       0.     351.9442   0.       0.     142.4969]


Solve the same LP using GLPK_MI

In [6]:
print(cvx.installed_solvers())
prob.solve(solver=cvx.GLPK_MI)
np.set_printoptions(precision=2,suppress=True)
print('Problem status: ' + str(prob.status));
if (prob.status == 'optimal'):
    print('Problem value: ' + str(prob.value));
    print('loan values: ')
    print(x.value.transpose())
    print('paper values: ')
    print(y.value.transpose())
    print('cash values: ')
    print(z.value.transpose())

['CVXOPT', 'ECOS', 'ECOS_BB', 'GLPK', 'GLPK_MI', 'GUROBI', 'OSQP', 'SCIPY', 'SCS']
Problem status: optimal
Problem value: 142.49694915254236
loan values: 
[-0.   50.98 -0.   -0.   -0.  ]
paper values: 
[150.    49.02 203.43]
cash values: 
[ -0.    -0.   351.94  -0.    -0.   142.5 ]


### Sensitivity analysis via basis

In [ ]:
# Formulate a standard form LP
n = (ntimes-1) + (ntimes-paper_period) + ntimes 
m = ntimes

xoffset = 0
yoffset = xoffset + (ntimes-1) 
zoffset = yoffset + (ntimes-paper_period) 

A = np.zeros((m,n));
# liability constraints
for i in range(ntimes):
    a = np.zeros(n)
    if (i<ntimes-1):
        a[i] = 1; # loan available
    if (i > 0):
        a[i-1] = -(1+loan_int/100) # loan has to repaid
    if (i < ntimes-paper_period):
        a[yoffset+i] = 1 # paper can be issued 
    if (i >= paper_period):
        a[yoffset + i - paper_period] = -(1+paper_int/100)
    if (i > 0):
        a[zoffset + i-1] = (1+rf_int/100)
    a[zoffset + i] = -1
    A[i,:] = a
    
b = np.array(liability).T
c = np.zeros((n,1))
c[n-1] = 1
    

In [ ]:
# Define the basis from the gurobi solution
tol = 1e-6
B = [i for i in range(ntimes-1) if x.value[i] > tol] 
B += [(yoffset + i) for i in range(ntimes-paper_period) if y.value[i]>tol]
B += [(zoffset + i) for i in range(ntimes) if z.value[i]>tol]
B = np.array(B)
print(B)

### Sensititive analysis with respect to the right handside

In [ ]:
# define A_B^{-1}
invA_B = np.linalg.inv(A[:,B])

# basic solution 
xB = invA_B.dot(b);

LB = -np.inf*np.ones((m,1))
UB = np.inf*np.ones((m,1))

for i in range(m):
    ei = np.zeros((m,1)); ei[i] = 1;
    d = invA_B.dot(ei);
    for j in range(m):
        if (float(d[j]>0)):
            LB[i] = max(LB[i], -float(xB[j])/float(d[j]))
        elif (float(d[j]<0)):
            UB[i] = min(UB[i], -float(xB[j])/float(d[j]))
    LB[i] += b[i]
    UB[i] += b[i]
    
np.set_printoptions(precision=4,suppress=True)
p = (((c[B]).T).dot(invA_B)).T

table = [[t+1, p[t], LB[t], b[t], UB[t]] for t in range(m)];
print(tabulate(table, headers=["Time","Shadow price", "RHS Low", "RHS", "RHS Up"], floatfmt=".4f")); 

### What is maximum interest rate would one still move liability from time 1 to time 6? And how much?

The first part, i.e at what maximum interest rate would it still be profitable to move liability, can be answered using the shadow prices. 
+ An infinitesimal decrease $\delta$ in the liablility at time $t=1$ increases the objective by $-p_0 \delta$,  
+ and the increase $(1+r)\delta$ at time $t=6$ decreases the objective by $(1+r)p_5\delta$
+ Move profitable if 
$$
-p_0 + (1+r)p_5 \geq 0 \quad \Rightarrow \quad r \leq \frac{p_0}{p_5} - 1
$$

However, the second part of the question, i.e. how much would one move, cannot be accurately using the information in the above table, because the table was created by assuming that only one component of the $b$ vector changes at a time. (Or, can it?)

So, what happens when multiple components change? One can apply the same methodology. Suppose
$$
b = b + \theta d
$$
for some $\theta \in \mathbb{R}$. The current basis will remain optimal provided
$$
x_B = A_B^{-1}b + \theta A_{B}^{-1} d \geq 0.
$$
One can use this inequality to compute upper and lower bounds on $\theta$. 

In this particular case
$$
d = \begin{bmatrix} -1 \\ 0 \\ \vdots \\ 0 \\ (1+r) \end{bmatrix}
$$

In [ ]:
month = 6;
rmax = (p[0]/p[month-1]-1)*100;
print('Maximum interest rate rmax = %0.3f%%' % (rmax))

# basic solution 
xB = invA_B.dot(b);

theta_low = -np.inf
theta_up = np.inf

d = np.zeros((m,1)); d[0] = -1; d[month-1] = (1+rmax/100); 
d = invA_B.dot(d);
for j in range(m):
    if (float(d[j]>0)):
        theta_low = max(theta_low, -float(xB[j])/float(d[j]))
    elif (float(d[j]<0)):
        theta_up = min(theta_up, -float(xB[j])/float(d[j]))
print('Maximum liability moved = %.3f' % theta_up)